In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import glob
import PIL.Image as Image
import torch.utils.data as data
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm

In [ ]:
im1_mean = [0.4042127201928427, 0.412576527722026, 0.42259604676925305, 0.43350413711051056, 0.443855690032959, 0.45140969585826785, 0.45334100553503703, 0.4468871853247354, 0.43043786530179756, 0.404718756014092, 0.37331931684981945, 0.3418279141576013,0.31582148226006085]
im2_mean = [0.41915108148831637, 0.42160734423110063, 0.423962258067611, 0.42601487354034223, 0.4275087523492551, 0.4281458628877129, 0.4276228290175893, 0.4257433505811504, 0.4224049007987228, 0.41766727063885856, 0.411706916417739, 0.4048067182780746,0.39733730345961626]
im3_mean = [0.44033469101140527, 0.4503952708250446, 0.46193364562084055, 0.4738397130390685, 0.48420565326379844, 0.49040465635549285, 0.48956380687421636, 0.47946231783341775, 0.45967343462004484, 0.4323601088249301, 0.4019184607310354, 0.37350198569667487,0.3512268327779888]

im1_std = [0.14732023241999434, 0.147809241675677, 0.14804508827283613, 0.1477574222750175, 0.1470682801149269, 0.146539035212084, 0.14675348492637386, 0.14775991989824996, 0.14848621292810835, 0.1465088569090591, 0.13925408874284476, 0.12590440973744257,0.10823487264730804]
im2_std = [0.12246606074712985, 0.12329658876446659, 0.1242021620079717, 0.12510900974078862, 0.12595538841389356, 0.12670439991891755, 0.12730158828399146, 0.12765988385551708, 0.12760264169433708, 0.1269505535408886, 0.12552824561659184, 0.12318955457930125,0.11984089703152391]
im3_std = [0.11690682253979083, 0.11736897240085645, 0.11786639327518504, 0.11859482672002022, 0.11987161521520828, 0.1221401983344635, 0.12578272075012656, 0.13025746011922526, 0.13357738663561808, 0.13312490121042017, 0.12725185198165137, 0.11619303606414197,0.10181700640611481]

calibrator1 = {}
calibrator2 = {}

for layer_idx in range(13):
    def calibration1(layer_data):
        mask = (layer_data >= 0.1) & (layer_data <= 0.9) 
        scaled_x = np.copy(layer_data)  
        scaled_x[mask] = ((layer_data[mask] - im2_mean[layer_idx])/im2_std[layer_idx]) * im1_std[layer_idx] + im1_mean[layer_idx] 
        return scaled_x
    
    def calibration2(layer_data):
        mask = (layer_data >= 0.1) & (layer_data <= 0.9) 
        scaled_x = np.copy(layer_data)  
        scaled_x[mask] = ((layer_data[mask] - im3_mean[layer_idx])/im3_std[layer_idx]) * im1_std[layer_idx] + im1_mean[layer_idx] 
        return scaled_x    

    calibrator1[layer_idx] = calibration1
    calibrator2[layer_idx] = calibration2

In [ ]:
import numba
import numpy as np
from PIL import Image
import torch
from matplotlib import pyplot as plt
from torch.utils.data import Dataset
from typing import Tuple, List
from torchvision.transforms import transforms
import cv2

class VesuviusTrainData(Dataset):
    _relative_sv_dir = "surface_volume"  # relative to the directory of train data

    def __init__(self,
                 dir_path: str = "train/1",
                 z_start: int = 0,
                 z_end: int = 64,
                 nucleus_shape: Tuple[int, int] = (16, 16),
                 hull_size: Tuple[int, int] = (64, 64),
                 compress_depth =  None,
                 give_indx = False,
                 calibrator = None,
                 boundarysize = None
                 ):
        self.dir_path = dir_path

        self.z_start = z_start
        self.z_end = z_end

        self.nucleus_shape = nucleus_shape
        self.hull_size = hull_size
        
        self.compress_depth = compress_depth
        self.give_indx = give_indx
        self.calibrator = calibrator
        self.boundarysize = boundarysize
        
        
        self._setup()

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index: int):
        i, j = self._get_pixel_from_index(self.indices[index])
        nucleus_height, nucleus_width = self.nucleus_shape

        if self.give_indx == False:
            if self.calibrator != None:
                return (torch.from_numpy(
                    self.calibrate(self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ])),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]))
            else:
                return (torch.from_numpy(
                    self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ]),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height])) 
        else:
            if self.calibrator != None:
                return (torch.from_numpy(
                    self.calibrate(self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ])),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]),
                   np.stack(([i+k for k in range(nucleus_width)],[
                    j+k for k in range(nucleus_height)]),axis=1))
            else:
                return (torch.from_numpy(
                    self.images[
                    :,
                    i - self._left_hull + self._left_pad_image:
                    i + nucleus_width + self._right_hull + self._left_pad_image,
                    j - self._top_hull + self._top_pad_image:
                    j + nucleus_height + self._bottom_hull + self._top_pad_image
                     ]),
                    torch.from_numpy(
                    self.inklabels[i: i + nucleus_width,
                    j: j + nucleus_height]),
                   np.stack(([i+k for k in range(nucleus_width)],[
                    j+k for k in range(nucleus_height)]),axis=1)) 
            
    def calibrate(self,arr):
        calibrated_arr = np.zeros_like(arr)

        for i in range(arr.shape[0]):
            calibration_func = self.calibrator[i]
            calibrated_arr[i, :, :] = calibration_func(arr[i, :, :])
        return calibrated_arr


    def _load_images(self, z_start=None,z_end=None):
        if z_end is None:
            z_end = self.z_end 
        if z_start is None:
            z_start = self.z_start
            
        num_images = z_end - z_start
        self.images = np.empty(
            (
                num_images,
                self.mask.shape[0] + self._left_pad_image + self._right_pad_image,
                self.mask.shape[1] + self._top_pad_image + self._bottom_pad_image
            ), 
            dtype=np.float16
        )
        for index, i in enumerate(range(z_start, z_end)):
            # noinspection PyTypeChecker
            image = np.array(Image.open(f"{self.dir_path}"
                                        f"/{self._relative_sv_dir}"
                                        f"/{i:02d}.tif"),
                             dtype=np.float32) / 65535.0
            
            image = image.astype(np.float16)
            
            image = np.pad(image, ((self._left_pad_mask + self._left_pad_image,
                                    self._right_pad_mask + self._right_pad_image),
                                   (self._top_pad_mask + self._top_pad_image,
                                    self._bottom_pad_mask + self._bottom_pad_image)),
                           'constant', constant_values=0)
            self.images[index, :, :] = image
            
    def _load_scale(self):
        current_stack_layer = self.z_start
        images = np.empty(
            (
                len(self.compress_depth),
                self.mask.shape[0] + self._left_pad_image + self._right_pad_image,
                self.mask.shape[1] + self._top_pad_image + self._bottom_pad_image
            ), 
            dtype=np.float16
        )
        for index,layer in enumerate(self.compress_depth):
            self._load_images(current_stack_layer, current_stack_layer + layer)
            current_stack_layer += layer
            layer_stack = self.images
            images[index,:,:] = np.mean(layer_stack,axis = 0)
            
        self.images = images
        
        
    def crop_boundary(self,mask, boundary_size):
        mask = mask.astype(np.uint8)
        mask_binary = cv2.threshold(mask, 0.5, 1, cv2.THRESH_BINARY)[1]

        contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        boundary_mask = np.ones_like(mask, dtype=np.uint8)

        cv2.drawContours(boundary_mask, contours, -1, (0), 100)

        kernel_size = int(boundary_size)  # Adjust the kernel size based on your needs
        kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
        eroded_boundary_mask = cv2.erode(boundary_mask, kernel, iterations=1)

        cropped_mask = mask * eroded_boundary_mask[:, :]

        return cropped_mask
    
    # Internal utility methods
    def _setup(self):
        # noinspection PyTypeChecker
        self.mask = np.array(Image.open(f"{self.dir_path}/mask.png"))
        if (self.boundarysize) != None:
            self.mask = self.crop_boundary(self.mask,self.boundarysize)
        # noinspection PyTypeChecker
        self.inklabels = np.array(Image.open(f"{self.dir_path}/inklabels.png"))
        self._granulate_mask()
        self._fill_new_mask()
        if self.compress_depth is None:
            self._load_images()
        else:
            self._load_scale()
        self._get_leftover_hull()

    def _granulate_mask(self):
        self._get_paddings()
        self.mask = np.pad(self.mask,
                           ((self._left_pad_mask, self._right_pad_mask),
                            (self._top_pad_mask, self._bottom_pad_mask)),
                           'constant', constant_values=0)
        self.indices = _find_labeled_nucleus(self.mask, self.nucleus_shape)
        self.inklabels = np.pad(self.inklabels, 
                                ((self._left_pad_mask, self._right_pad_mask),
                                 (self._top_pad_mask, self._bottom_pad_mask)),
                                'constant', 
                                constant_values=0)

    def _get_paddings(self):
        width, height = self.mask.shape
        nucleus_width, nucleus_height = self.nucleus_shape
        hull_width, hull_height = self.hull_size
        self._left_pad_mask = (width % nucleus_width) // 2
        self._right_pad_mask = width % nucleus_width - self._left_pad_mask
        self._top_pad_mask = (height % nucleus_height) // 2
        self._bottom_pad_mask = height % nucleus_height - self._top_pad_mask

        self._left_pad_image = (hull_width - nucleus_width) // 2
        self._right_pad_image = hull_width - nucleus_width - self._left_pad_image
        self._top_pad_image = (hull_height - nucleus_height) // 2
        self._bottom_pad_image = hull_height - nucleus_height - self._top_pad_image

    def _fill_new_mask(self):
        nucleus_width, nucleus_height = self.nucleus_shape
        self.mask = np.zeros_like(self.mask)
        for index in self.indices:
            i, j = self._get_pixel_from_index(index)
            self.mask[
                i * nucleus_width: (i + 1) * nucleus_width,
                j * nucleus_height: (j + 1) * nucleus_height
            ] = 1

    def _get_pixel_from_index(self, index: int):
        mask_width, mask_height = self.mask.shape
        nucleus_width, nucleus_height = self.nucleus_shape
        height = mask_height // nucleus_height
        return index // height * nucleus_height, index % height * nucleus_width

    def _get_leftover_hull(self):
        nucleus_width, nucleus_height = self.nucleus_shape
        hull_width, hull_height = self.hull_size
        self._left_hull = (hull_width - nucleus_width) // 2
        self._right_hull = hull_width - nucleus_width - self._left_hull
        self._top_hull = (hull_height - nucleus_height) // 2
        self._bottom_hull = hull_height - nucleus_height - self._top_hull


@numba.njit()
def _find_labeled_nucleus(mask, nuclues_shape):
    width, height = mask.shape
    nucleus_width, nucleus_height = nuclues_shape

    indices = - np.ones((width // nucleus_width) * (height // nucleus_height), dtype=np.int32)
    current_index = 0
    for i in range(width // nucleus_width):
        for j in range(height // nucleus_height):
            if np.sum(
                    mask[i * nucleus_width: (i + 1) * nucleus_width,
                         j * nucleus_height: (j + 1) * nucleus_height]
                      ) > 0:
                indices[current_index] = i * (height // nucleus_height) + j
                current_index += 1
    return indices[indices != -1]

In [ ]:
nucleus_shape = (8,8)
hull_size = (24,24)
z_start = 22
z_end = 35
boundarysize = 700

In [ ]:
data1 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/1"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,boundarysize = boundarysize+200)

In [ ]:
data2 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/2"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,boundarysize = boundarysize+500)

In [ ]:
import numpy as np
import cv2
from torch.utils.data import DataLoader


def bm3d_denoise(patch, block_size=10, search_window_size=16, sigma=20):
    # Step 2: Block matching and collaborative filtering
    denoised_patch = np.zeros_like(patch)
    height, width = patch.shape[:2]
    blocks_per_row = width // block_size
    blocks_per_col = height // block_size
    
    for j in range(blocks_per_col):
        for i in range(blocks_per_row):
            # Extract the current block
            block = patch[j*block_size:(j+1)*block_size, i*block_size:(i+1)*block_size]
            
            # Define the search window boundaries
            window_start_x = max(0, i*block_size - search_window_size)
            window_end_x = min((i+1)*block_size + search_window_size, width)
            window_start_y = max(0, j*block_size - search_window_size)
            window_end_y = min((j+1)*block_size + search_window_size, height)
            
            # Extract the search window
            search_window = patch[window_start_y:window_end_y, window_start_x:window_end_x]
            
            # Calculate Euclidean distances between the block and search window blocks
            distances = np.sum(np.square(block - search_window.reshape(-1, block_size, block_size)), axis=(1, 2))
            
            # Sort the distances and select similar blocks
            sorted_indices = np.argsort(distances)
            similar_blocks_indices = sorted_indices[:10]  # Choose the top 10 similar blocks
            
            # Collaborative filtering
            similar_blocks = search_window[similar_blocks_indices]
            weights = np.exp(-distances[similar_blocks_indices] / (2 * sigma**2))
            weighted_sum = np.sum(similar_blocks * weights[:, np.newaxis, np.newaxis], axis=0)
            sum_of_weights = np.sum(weights)
            filtered_block = weighted_sum / sum_of_weights
            
            # Update the denoised patch
            denoised_patch[j*block_size:(j+1)*block_size, i*block_size:(i+1)*block_size] = filtered_block
    
    return denoised_patch

train_loader = DataLoader(data1, batch_size = 1, shuffle=True)

output_patches = []
output_targs = []

for input_patch, targ,idx in tqdm(train_loader):
    denoised_patch = bm3d_denoise(input_patch)
    output_patches.append(denoised_patch)
    output_targs.append(targ)


In [ ]:
train_loader = DataLoader(data2, batch_size = 1, shuffle=True)
output_patches2 = []
output_targs2 = []

for input_patch, targ,idx in tqdm(train_loader):
    denoised_patch = bm3d_denoise(input_patch)
    output_patches2.append(denoised_patch)
    output_targs2.append(targ)

In [ ]:
from torch.utils.data import TensorDataset

tar = torch.stack(output_targs)
tar2 = torch.stack(output_targs2)
dataset1 = TensorDataset(torch.tensor(np.array(output_patches)),torch.tensor(tar))
dataset2 = TensorDataset(torch.tensor(np.array(output_patches2)),torch.tensor(tar2))

In [ ]:
from torch.utils.data import ConcatDataset
merged_dataset = ConcatDataset([dataset1, dataset2])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

class RandomNoise(object):
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.std + self.mean
        return tensor + torch.abs(noise)
    
def rotate_image(data,target):
    degrees = np.random.choice([90, 180, 270])
    rotated_data = np.rot90(data, k=degrees // 90, axes= (-2, -1)).copy()
    rotated_target = np.rot90(target, k=degrees // 90, axes=(-2, -1)).copy()
    return torch.tensor(rotated_data),torch.tensor(rotated_target)

def random_shift(data,target, max_shift=2):
    x_shift = np.random.randint(-max_shift, max_shift)
    y_shift = np.random.randint(-max_shift, max_shift)
    z_shift = np.random.randint(-max_shift, max_shift)
    shifted_data = torch.roll(data, shifts=(x_shift, y_shift,z_shift), dims=(-3,-2, -1))
    shifted_target = torch.roll(target, shifts=(x_shift, y_shift), dims=(-2, -1))
    
    return shifted_data,shifted_target

class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=(3,3,3), padding=1)
        self.bn1 = nn.BatchNorm3d(16)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=(3,2,2), padding=1)
        self.bn2 = nn.BatchNorm3d(32)
        self.conv3 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3,1,1), padding=1)
        self.bn3 = nn.BatchNorm3d(64)
        self.fc = nn.Linear(1024  , 64)
        
        self.saved_loss = []
        self.saved_epoch = []
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool3d(x, kernel_size=2)
        x = x.view(x.size(0), -1)  
        x = self.fc(x)
        return x
    
    def augment(self, data, target):
        batch_size = data.size(0)
        rands = torch.rand(batch_size)
        flip1 = transforms.RandomApply([transforms.RandomHorizontalFlip()], p=1)
        flip2 = transforms.RandomApply([transforms.RandomVerticalFlip()], p=1)
        noise = transforms.RandomApply([RandomNoise(mean=0, std=0.005)], p=1)
        data_list = []
        tar_list = []

        for i in range(batch_size):
            if torch.sum(target[i]) == 0:
                random_number = random.random() 
                if random_number > 0.4:
                    continue
            
            if rands[i] < 0.1:
                dat, tar= rotate_image(data[i],target[i])
                data_list.append(dat)
                tar_list.append(tar)
            elif rands[i] < 0.3:
                dat,tar = random_shift(data[i],target[i]) 
                data_list.append(dat)
                tar_list.append(tar)
            else:
                data_list.append(data[i])
                tar_list.append(target[i])
                
        return  torch.stack(data_list), torch.stack(tar_list)


    def fit(self, train_data,optimizer, criterion, num_epochs,verbose=False): 
        train_loader = DataLoader(train_data, batch_size = 512, shuffle=True)
        for epoch in range(num_epochs):
            loss_list = []
            f1_list = []
            for batch_idx, (data, target, idxs) in tqdm(enumerate(train_loader)):
                data = data.to(torch.float)
                data, target = self.augment(data,target)
                inputs = data.view(data.shape[0],1, *data.shape[1:])
                inputs = inputs.to(device)
                output = (self(inputs))
                target = target.to(device)
                target = target.to(torch.float)
                optimizer.zero_grad()
                loss = criterion(output, target.view(target.shape[0], -1))
                loss_list.append(loss.item())
                loss.backward()
                optimizer.step()
                if batch_idx % 100 == 0:
                    print(np.mean(loss_list))
                
            if verbose == True:
                epoch_loss = np.mean(loss_list)
                self.save_loss(epoch_loss,epoch)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
            
                
    def save_loss(self,epoch_loss,epoch):
        self.saved_loss.append(epoch_loss)
        if len(self.saved_epoch) != 0:
            if self.saved_epoch[-1] > epoch:
                self.saved_epoch.append(self.saved_epoch[-1]+1)
            else:
                 self.saved_epoch.append(epoch)
        else:
            self.saved_epoch.append(epoch)

In [ ]:
model = CNN3D()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
#model.load_state_dict(torch.load("/kaggle/input/3dweights/new3d-2.pth", map_location=torch.device('cpu')))
model.to(device)

In [ ]:
num_epochs = 3
model.fit(merged_dataset,optimizer,criterion,num_epochs,verbose = True)
#torch.save(model.state_dict(), 'new_3dweights.pth')

In [ ]:
torch.save(model.state_dict(), 'again.pth')

In [ ]:
data3 = VesuviusTrainData(
    dir_path="/kaggle/input/vesuvius-challenge-ink-detection/train/3"\
    ,nucleus_shape = nucleus_shape,hull_size=hull_size, z_start=z_start
    , z_end=z_end,give_indx=True,calibrator=calibrator2)

In [ ]:
train_loader = DataLoader(data3, batch_size = 1, shuffle=True)

output_patches3 = []
output_targs3 = []

for input_patch, targ,idx in tqdm(train_loader):
    denoised_patch = bm3d_denoise(input_patch)
    output_patches3.append(denoised_patch)
    output_targs3.append(targ)

In [ ]:
tar3 = torch.stack(output_targs3)

dataset3 = TensorDataset(torch.tensor(np.array(output_patches3)),torch.tensor(tar3))

In [ ]:
score = []
preds = []
targs = []
val_loader =DataLoader(dataset3, batch_size = 512,shuffle=True)
for data, target,idxs in tqdm((val_loader)):
    data = data.to(torch.float)
    output = model(data.view(data.shape[0],1,*data.shape[1:]).to(device))
    preds.append((nn.Sigmoid()(output)).detach().cpu().numpy())
    targs.append(target)


In [ ]:
class Fscore:
    def __init__(self):
        self.epsilon = 1e-7
        self.tp = 0
        self.fp = 0
        self.fn = 0
    
    def accumulate(self, y_true, y_pred):
        # Flatten the input arrays to 1D
        y_true = np.ravel(y_true)
        y_pred = np.ravel(y_pred)
        
        # Update true positives, false positives, and false negatives
        self.tp += np.sum((y_true == 1) & (y_pred == 1))
        self.fp += np.sum((y_true == 0) & (y_pred == 1))
        self.fn += np.sum((y_true == 1) & (y_pred == 0))
    
    def get_score(self, beta=1):
       # Calculate precision, recall, and F-score
        precision = self.tp / (self.tp + self.fp + self.epsilon)
        recall = self.tp / (self.tp + self.fn + self.epsilon)
        f_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall + self.epsilon)
    
        return f_score

In [ ]:
import math
import numpy as np

def round_elements(arr, threshold):
    rounded_arr = np.array([])
    for elem in (arr):
        elem = torch.tensor(elem.reshape(8*8))

        rounded_elem = np.zeros_like(elem)        
        rounded_arr = np.append(rounded_arr,torch.where(elem >= threshold, torch.ceil(elem), torch.floor(elem)))
    return rounded_arr

In [ ]:
thrsh = 0.5

score = Fscore()
for p,t in tqdm(zip(preds,targs)):
    score.accumulate(t,round_elements(p,thrsh))

print(score.get_score(beta=0.5))

In [ ]:
flatten = np.concatenate(preds).flatten()
N= len(flatten)
tn = N - (score.tp + score.fn + score.fp)

print(score.tp)
print(score.fn)
print(score.fp)

print(score.tp/(score.tp+score.fp))
print(score.tp/(score.tp+score.fn))
print((score.tp+score.fp)/(tn+score.fn))

In [ ]:
picture = np.zeros((8298, 6452))

@numba.njit()
def insert_snippet(full_picture,indx_matrix,sheight,swidth,smatrix):
    height = indx_matrix[0]
    width = indx_matrix[1]
    for i in range(sheight):
        for j in range(swidth):
            full_picture[height[i],width[j]] = smatrix[i,j]
    return full_picture

pics_loader = DataLoader(data3, batch_size=1)
for batch_idx, (x_data, target, indx_arr) in tqdm(enumerate(pics_loader)):
    x_data = x_data.view(1,*x_data.shape)
    x_data = x_data.to(torch.float)
    x_data = torch.tensor((x_data).to(device))
    pred =  torch.sigmoid(model(x_data))
    pred_cpu = pred.detach().cpu().numpy()
    picture = insert_snippet(picture, indx_arr[0, :].T.numpy(), 8,8, pred_cpu.reshape(8, 8))
    if batch_idx > 400000:
        break


In [ ]:
def round_array_threshold(array, threshold):
    rounded_array = np.where(array >= threshold, np.ceil(array), np.floor(array))
    return rounded_array

In [ ]:
rounded_array = round_array_threshold(picture,0.5)
plt.imshow(rounded_array)
plt.colorbar()

In [ ]:
print(score.tp)
print(score.fn)
print(score.fp)

print(score.tp/(score.tp+score.fp))
print(score.tp/(score.tp+score.fn))

In [ ]:
print((score.tp+score.fp)/(tn+score.fn))

In [ ]:
flatten = []
for p in preds:
    flatten.append(p.flatten())

In [ ]:
flatten = np.concatenate(preds).flatten()

In [ ]:
N= len(flatten)
tn = N - (score.tp + score.fn + score.fp)

In [ ]:
(score.tp + tn) / (score.tp + score.fn + score.fp + tn)